# File and libraries

In [38]:
import pandas as pd
import numpy as np


filepath=r'google_merch_store_raw_merge.csv'

sample_filepath=r'google_merch_store_ga4_merch_flattened_raw_000000000000.csv'



file_tag = "ga4_merch_store"

# DSLabs functions

In [39]:
%run "dslabs_functions.py"


# data functions

In [40]:
%run "data_functions.py"


data_functions lodaded


# Load

In [41]:
# test_data=True
test_data=False

if test_data==True:
    # Specify the chunk size (e.g., load 1000 lines at a time)
    chunk_size = 50000

    # Initialize an empty list to store the chunks
    chunks = []

    # Load the first chunk (or any number of chunks you want)
    for chunk in pd.read_csv(filepath, chunksize=chunk_size, parse_dates=["event_date"],date_format="%Y-%m-%d"):
        chunks.append(chunk)
        # Break after the first chunk to load only a sample
        break

    # Concatenate the chunks into a single DataFrame (optional)
    data = pd.concat(chunks)

    # data=pd.read_csv(filepath)

    data['event_timestamp'] = pd.to_datetime(data['event_timestamp'], infer_datetime_format=True)
    data['event_date'] = pd.to_datetime(data['event_date'], infer_datetime_format=True)
 
    # # sort values by session id so that sessions are ordered and not excluded from dataset
    # data = data.sort_values(by=['session_id', 'event_timestamp'], ascending=True)
 
    # # 1% sample
    # data=data.sample(frac=0.01, replace=False)

    

else:
    data=pd.read_csv(filepath)

    data['event_timestamp'] = pd.to_datetime(data['event_timestamp'], infer_datetime_format=True)
    data['event_date'] = pd.to_datetime(data['event_date'], infer_datetime_format=True)

    # sort values by session id so that sessions are ordered and not excluded from dataset
    # data = data.sort_values(by=['session_id', 'event_timestamp'], ascending=True)
 
    # 10% sample
    data=data.sample(frac=0.1, replace=False)

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 776597 entries, 3674016 to 4539179
Data columns (total 48 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   event_date                       776597 non-null  datetime64[ns]
 1   session_id                       776597 non-null  int64         
 2   user_pseudo_id                   776597 non-null  float64       
 3   event_name                       776597 non-null  object        
 4   event_timestamp                  776597 non-null  datetime64[ns]
 5   page_location                    776597 non-null  object        
 6   page_title                       774522 non-null  object        
 7   device_category                  776597 non-null  object        
 8   device_mobile_brand_name         776597 non-null  object        
 9   device_mobile_model_name         776597 non-null  object        
 10  device_mobile_marketing_name     776597 no

In [42]:
data.shape

(776597, 48)

# class target column creation


we want to classify if that hit is from a returning or new user so we need to group the ga_session_number by 1 or more than 1 session. in this case, new user (ga_session_number = 1) will be 0 and returning user will be more than 1 (ga_session_number > 1)

In [43]:
data['returning_user'] = data['ga_session_number'].apply(lambda x: 0 if x == 1 else 1)

data=data.drop(['ga_session_number'],axis=1) # now we do not need it anymore. remove it


# date column creation

## event_date

In [44]:
data['event_date'] = pd.to_datetime(data['event_date'], infer_datetime_format=True)


# create year, quarter, month, day number of week, weekend/weekday based on event_date column

# Create new columns
data['year'] = data['event_date'].dt.year
data['quarter'] = data['event_date'].dt.quarter
data['month'] = data['event_date'].dt.month
data['day'] = data['event_date'].dt.day
data['day_of_week'] = data['event_date'].dt.day_name()  
data['day_of_year'] = data['event_date'].dt.dayofyear  # Day of the year
data['week_number'] = data['event_date'].dt.isocalendar().week  # ISO week number

data['day_of_week_nr'] = data['event_date'].dt.weekday  # Monday=0, Sunday=6
data['is_weekend'] = data['day_of_week_nr'].apply(lambda x: 1 if x >= 5 else 0)  # 1 for weekend, 0 for weekday
data.drop(['day_of_week_nr'],axis=1) # already symbolic. not needed


,event_date,session_id,user_pseudo_id,event_name,event_timestamp,page_location,page_title,device_category,device_mobile_brand_name,device_mobile_model_name,...,promotion_name,returning_user,year,quarter,month,day,day_of_week,day_of_year,week_number,is_weekend
3674016,2020-11-20,2286967592,1.426511e+07,view_item,1970-01-19 14:04:13.175600046,https://shop.googlemerchandisestore.com/Google...,Apparel | Google Merchandise Store,desktop,Google,Chrome,...,(not set),0,2020,4,11,20,Friday,325,47,0
1841984,2020-12-21,9995254576,6.507727e+06,view_item,1970-01-19 14:49:45.376155130,https://shop.googlemerchandisestore.com/Google...,Android | Shop by Brand | Google Merchandise S...,mobile,Samsung,<Other>,...,(not set),1,2020,4,12,21,Monday,356,52,0
2336114,2020-12-15,759046623,4.624183e+06,view_item,1970-01-19 14:40:23.790828995,https://shop.googlemerchandisestore.com/Google...,Sale | Google Merchandise Store,desktop,Apple,Safari,...,(not set),0,2020,4,12,15,Tuesday,350,51,0
3473771,2020-12-15,1788799216,6.451660e+07,view_item,1970-01-19 14:39:52.824057082,https://shop.googlemerchandisestore.com/Google...,Men's / Unisex | Apparel | Google Merchandise ...,desktop,Apple,Safari,...,(not set),1,2020,4,12,15,Tuesday,350,51,0
2878384,2020-12-08,5356421185,4.901400e+07,view_item,1970-01-19 14:29:59.206427173,https://shop.googlemerchandisestore.com/Google...,Sale | Google Merchandise Store,desktop,Google,Chrome,...,(not set),0,2020,4,12,8,Tuesday,343,50,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6589870,2020-12-28,8962365735,5.910727e+07,view_item,1970-01-19 14:59:28.936632820,https://shop.googlemerchandisestore.com/Google...,YouTube | Shop by Brand | Google Merchandise S...,desktop,Google,Chrome,...,(not set),1,2020,4,12,28,Monday,363,53,0
3310786,2020-12-08,3437939227,7.260568e+07,user_engagement,1970-01-19 14:30:06.368505926,https://shop.googlemerchandisestore.com/basket...,Shopping Cart,mobile,Apple,iPhone,...,NaN,1,2020,4,12,8,Tuesday,343,50,0
5911426,2020-12-11,7271906290,7.257732e+06,view_item,1970-01-19 14:35:09.215763407,https://shop.googlemerchandisestore.com/Google...,Men's / Unisex | Apparel | Google Merchandise ...,desktop,Google,ChromeBook,...,(not set),0,2020,4,12,11,Friday,346,50,0
5119277,2020-12-23,438337067,2.312554e+06,view_item,1970-01-19 14:51:33.738689033,https://shop.googlemerchandisestore.com/store....,Home,desktop,Google,Chrome,...,(not set),1,2020,4,12,23,Wednesday,358,52,0


# event_timestamp

In [45]:
# Convert 'event_timestamp' from microseconds to datetime
data['event_timestamp'] = pd.to_datetime(data['event_timestamp'], unit='us')

# Create new columns
data['hour_of_day'] = data['event_timestamp'].dt.hour  # Extract hour (0-23)
data['minute'] = data['event_timestamp'].dt.minute  # Extract minute
data['hour_minute_fraction'] = data['hour_of_day'] + data['minute'] / 60  # Hour + fraction of minute

# Categorize the time of day
def categorize_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

data['time_of_day'] = data['hour_of_day'].apply(categorize_time_of_day)

# column drop

### low value or high null count columns

In [46]:
data=data.drop(['debug_mode','device_is_limited_ad_tracking','device_mobile_marketing_name','geo_metro','traffic_source_name','page_referrer','entrances'],axis=1)

### ecommerce specific columns

In [47]:
data=data.drop([
 'ecommerce_total_item_quantity', 
 'ecommerce_purchase_revenue',            
 'ecommerce_shipping_value',              
 'ecommerce_tax_value',                   
 'ecommerce_unique_items',               
 'ecommerce_transaction_id',              
 'item_id',                               
 'item_name',                             
 'item_brand',                            
 'item_variant',                          
 'item_category',                         
 'price',                                 
 'quantity',                              
 'item_revenue',                          
 'item_list_index',                       
 'promotion_name'],axis=1)

In [48]:
summary5 = data.describe(include="all")

summary5

,event_date,session_id,user_pseudo_id,event_name,event_timestamp,page_location,page_title,device_category,device_mobile_brand_name,device_mobile_model_name,...,day,day_of_week,day_of_year,week_number,day_of_week_nr,is_weekend,hour_of_day,minute,hour_minute_fraction,time_of_day
count,776597,7.765970e+05,7.765970e+05,776597,776597,776597,774522,776597,776597,776597,...,776597.000000,776597,776597.000000,776597.0,776597.000000,776597.000000,776597.000000,776597.000000,776597.000000,776597
unique,NaN,NaN,NaN,17,NaN,1462,489,3,8,10,...,NaN,7,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,1
top,NaN,NaN,NaN,view_item,NaN,https://shop.googlemerchandisestore.com/Google...,Home,desktop,Apple,Chrome,...,NaN,Tuesday,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,Afternoon
freq,NaN,NaN,NaN,289012,NaN,72758,124838,451937,330337,215346,...,NaN,133990,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,776597
mean,2020-12-14 05:45:47.984347136,4.987490e+09,2.617312e+08,NaN,1970-01-19 14:39:28.019073926,NaN,NaN,NaN,NaN,NaN,...,15.213157,NaN,250.030945,37.103093,2.718685,0.207052,14.141445,30.479290,14.649433,NaN
min,2020-11-01 00:00:00,6.412000e+03,1.000300e+06,NaN,1970-01-19 13:36:28.843867564,NaN,NaN,NaN,NaN,NaN,...,1.000000,NaN,1.000000,1.0,0.000000,0.000000,13.000000,0.000000,13.600000,NaN
25%,2020-11-26 00:00:00,2.503411e+09,5.805969e+06,NaN,1970-01-19 14:12:49.998468615,NaN,NaN,NaN,NaN,NaN,...,8.000000,NaN,29.000000,4.0,1.000000,0.000000,14.000000,19.000000,14.200000,NaN
50%,2020-12-11 00:00:00,4.992569e+09,2.296957e+07,NaN,1970-01-19 14:34:47.849807588,NaN,NaN,NaN,NaN,NaN,...,15.000000,NaN,329.000000,48.0,3.000000,0.000000,14.000000,32.000000,14.566667,NaN
75%,2021-01-04 00:00:00,7.486894e+09,6.172445e+07,NaN,1970-01-19 15:09:51.914335815,NaN,NaN,NaN,NaN,NaN,...,22.000000,NaN,345.000000,50.0,4.000000,0.000000,15.000000,43.000000,15.150000,NaN
max,2021-01-31 00:00:00,9.999997e+09,9.985207e+09,NaN,1970-01-19 15:48:57.594824131,NaN,NaN,NaN,NaN,NaN,...,31.000000,NaN,366.000000,53.0,6.000000,1.000000,15.000000,59.000000,15.800000,NaN


# row drop (from values)

## specific event_names



In [49]:
data['event_name'].value_counts()

event_name
view_item              289012
page_view              135501
user_engagement        105651
add_to_cart             66859
scroll                  49473
session_start           35516
select_item             32504
first_visit             25678
view_promotion          19160
begin_checkout           8592
view_search_results      2587
add_shipping_info        1955
purchase                 1569
add_payment_info         1428
select_promotion          933
click                     126
view_item_list             53
Name: count, dtype: int64

### event_name insights
We want to classify a user by its interactions with the website so we want to exclude some actions that may also be biased by incorrect ga4 tracking namely:
- session_start
- first_visit
- click (low event count)
- view_item_list (may not be triggered by user interaction)

In [50]:
# List of values to drop
events_to_drop = ['session_start', 'first_visit','click','view_item_list']

# drop events from list
data = data[~data['event_name'].isin(events_to_drop)]


data['event_name'].value_counts()

event_name
view_item              289012
page_view              135501
user_engagement        105651
add_to_cart             66859
scroll                  49473
select_item             32504
view_promotion          19160
begin_checkout           8592
view_search_results      2587
add_shipping_info        1955
purchase                 1569
add_payment_info         1428
select_promotion          933
Name: count, dtype: int64

# replace (not set) with null

we will handle these later but these are actually null values

In [51]:
data.replace('(not set)', np.nan, inplace=True)

# Engagement time msec
- https://support.google.com/analytics/answer/11109416?hl=en
- is it null?

In [52]:
data['engagement_time_msec'] = data['engagement_time_msec'].fillna(0)

# geo columns

## move subcontinent

In [53]:
# Move 'geo_sub_continent' from index 16 after continent

# Pop 'geo_sub_continent' column
geo_sub_continent = data.pop('geo_sub_continent')  # Remove the column

# Insert 'geo_sub_continent' at index 16
data.insert(16, 'geo_sub_continent', geo_sub_continent)


# Display the reordered DataFrame
print("\nReordered DataFrame:")
data.info()


Reordered DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 715224 entries, 3674016 to 4539179
Data columns (total 38 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   event_date                       715224 non-null  datetime64[ns]
 1   session_id                       715224 non-null  int64         
 2   user_pseudo_id                   715224 non-null  float64       
 3   event_name                       715224 non-null  object        
 4   event_timestamp                  715224 non-null  datetime64[ns]
 5   page_location                    715224 non-null  object        
 6   page_title                       713787 non-null  object        
 7   device_category                  715224 non-null  object        
 8   device_mobile_brand_name         715224 non-null  object        
 9   device_mobile_model_name         715224 non-null  object        
 10  device_operating_sys

## geo_region and geo_city

In [54]:
null_countries_df=data.groupby(['geo_country']).agg(
    null_count=('geo_region', lambda x: x.isna().sum())
).sort_values('null_count',ascending=False).reset_index()


countries_with_nulls = null_countries_df[null_countries_df['null_count'] > 0]['geo_country'].tolist()

print(countries_with_nulls)
# will retrieve the most populated cities of these countries to use as fill method

['Singapore', 'Hong Kong', 'United States', 'Peru', 'China', 'Russia', 'Japan', 'Mexico', 'Turkey', 'Bangladesh', 'Indonesia', 'Ireland', 'Colombia', 'Vietnam', 'Thailand', 'Romania', 'Ukraine', 'Portugal', 'Switzerland', 'Norway', 'Spain', 'Dominican Republic', 'Iraq', 'Finland', 'Sweden', 'India', 'Italy', 'Brazil', 'Serbia', 'Slovakia', 'Uruguay', 'Malaysia', 'Czechia', 'Egypt', 'Austria', 'South Korea', 'Hungary', 'Germany', 'Greece', 'United Arab Emirates', 'Chile', 'Morocco', 'Philippines', 'New Zealand', 'Poland', 'Palestine', 'Armenia', 'Argentina', 'Slovenia', 'Kuwait', 'North Macedonia', 'Netherlands', 'United Kingdom', 'Algeria', 'Bulgaria', 'Croatia', 'Bahrain', 'Cyprus', 'Australia', 'Nigeria', 'Panama', 'South Africa', 'Venezuela', 'Albania', 'Pakistan', 'Costa Rica', 'Puerto Rico', 'Bolivia', 'Azerbaijan', 'Luxembourg', 'Sri Lanka', 'Canada', 'Saudi Arabia', 'Lithuania', 'Jamaica', 'Paraguay', 'Kazakhstan', 'Malta', 'Macao', 'Ecuador', 'Kosovo', 'Trinidad & Tobago', 'Hon

In [55]:
most_populated_cities = {
    'Singapore': 'Singapore',
    'Hong Kong': 'Hong Kong',
    'Peru': 'Lima',
    'Dominican Republic': 'Santo Domingo',
    'Palestine': 'Gaza',
    'Iraq': 'Baghdad',
    'Serbia': 'Belgrade',
    'Uruguay': 'Montevideo',
    'North Macedonia': 'Skopje',
    'Cyprus': 'Nicosia',
    'Bahrain': 'Manama',
    'Armenia': 'Yerevan',
    'Panama': 'Panama City',
    'Azerbaijan': 'Baku',
    'Venezuela': 'Caracas',
    'Albania': 'Tirana',
    'Luxembourg': 'Luxembourg',
    'Jamaica': 'Kingston',
    'Macao': 'Macau',
    'Bolivia': 'Santa Cruz de la Sierra',
    'Trinidad & Tobago': 'Chaguanas',
    'Honduras': 'Tegucigalpa',
    'Paraguay': 'Asuncion',
    'Morocco': 'Casablanca',
    'Kosovo': 'Pristina',
    'Slovakia': 'Bratislava',
    'Malta': 'Birkirkara',
    'New Zealand': 'Auckland',
    'Bulgaria': 'Sofia',
    'Bangladesh': 'Dhaka',
    'Nigeria': 'Lagos'
}



data['geo_region'] = data['geo_region'].fillna(data['geo_country'].map(most_populated_cities))
data['geo_city'] = data['geo_city'].fillna(data['geo_country'].map(most_populated_cities))

In [56]:
null_regions_df=data.groupby(['geo_region']).agg(
    null_count=('geo_region', lambda x: x.isna().sum())
).sort_values('null_count',ascending=False).reset_index()

null_regions_df


,geo_region,null_count
0,Abruzzo,0
1,Porto District,0
2,Podkarpackie Voivodeship,0
3,Piedmont,0
4,Pichincha,0
...,...,...
390,Grand Est,0
391,Giza Governorate,0
392,Georgia,0
393,Geneva,0


In [57]:
null_region_cities_df=data.groupby(['geo_region']).agg(
    null_count=('geo_city', lambda x: x.isna().sum())
).sort_values('null_count',ascending=False).reset_index()


region_cities_with_nulls = null_region_cities_df[null_region_cities_df['null_count'] > 0]['geo_region'].tolist()


region_cities_with_nulls

['California',
 'Florida',
 'England',
 'Texas',
 'New Jersey',
 'Pennsylvania',
 'New York',
 'Ohio',
 'Michigan',
 'Taipei City',
 'Georgia',
 'Ontario',
 'Illinois',
 'Massachusetts',
 'Virginia',
 'North Carolina',
 'Quebec',
 'Maryland',
 'Connecticut',
 'Ile-de-France',
 'Wisconsin',
 'Missouri',
 'Indiana',
 'Washington',
 'Colorado',
 'Utah',
 'British Columbia',
 'South Carolina',
 'Minnesota',
 'Alberta',
 'Tennessee',
 'New Taipei City',
 'Catalonia',
 'Oregon',
 'Alabama',
 'North Rhine-Westphalia',
 'State of Sao Paulo',
 'Bavaria',
 'Flanders',
 'Louisiana',
 'Tokyo',
 'Baden-Wurttemberg',
 'Kentucky',
 'Iowa',
 'Mississippi',
 'Maharashtra',
 'Auvergne-Rhone-Alpes',
 'Arizona',
 'Taiwan Province',
 'Arkansas',
 'Oklahoma',
 'Lombardy',
 'Kansas',
 'Gyeonggi-do',
 'South Holland',
 'North Holland',
 'Emilia-Romagna',
 'Andalusia',
 'Kaohsiung City',
 'Tamil Nadu',
 'Wales',
 'West Bengal',
 'New Hampshire',
 'Taichung City',
 'Hauts-de-France',
 'Gujarat',
 'West Virginia

In [58]:
most_populated_cities_by_region = {
    'Taipei City': 'Taipei',
    'New Taipei City': 'New Taipei',
    'Taichung City': 'Taichung',
    'Kaohsiung City': 'Kaohsiung',
    'West Virginia': 'Charleston',
    'Delaware': 'Wilmington',
    'State of Mexico': 'Ecatepec',
    'Buenos Aires Province': 'La Plata',
    'Montana': 'Billings',
    'Wallonia': 'Liège',
    'Alaska': 'Anchorage',
    'Grand Est': 'Strasbourg',
    'Henan': 'Zhengzhou',
    'Hawalli Governorate': 'Hawalli',
    'Gelderland': 'Arnhem',
    'Algiers Province': 'Algiers',
    'North Dakota': 'Fargo',
    'Vermont': 'Burlington',
    'Idaho': 'Boise',
    'Canary Islands': 'Las Palmas',
    'Castile-La Mancha': 'Toledo',
    'Lublin Voivodeship': 'Lublin',
    'Calabarzon': 'Cavite City',
    'Saxony': 'Dresden',
    'Castile and Leon': 'Valladolid',
    'State of Bahia': 'Salvador',
    'Limburg': 'Hasselt',
    'Tainan City': 'Tainan',
    'Moscow Oblast': 'Khimki',
    'Chiba': 'Chiba',
    'Prince Edward Island': 'Charlottetown',
    'Overijssel': 'Zwolle',
    'Haifa District': 'Haifa',
    'Liguria': 'Genoa',
    'Northern Ireland': 'Belfast',
    'Rhineland-Palatinate': 'Mainz',
    'Sverdlovsk Oblast': 'Yekaterinburg',
    'Region of Southern Denmark': 'Odense',
    'Decentralized Administration of Peloponnese, Western Greece and the Ionian': 'Patras',
    'Amman Governorate': 'Amman',
    'Silesian Voivodeship': 'Katowice',
    'Asturias': 'Oviedo',
    'Galicia': 'Santiago de Compostela',
    'Hokkaido': 'Sapporo',
    'Quintana Roo': 'Cancún',
    'Centre-Val de Loire': 'Orléans',
    'Tamaulipas': 'Reynosa',
    'State of Santa Catarina': 'Florianópolis',
    'Saitama': 'Saitama',
    'Sicily': 'Palermo',
    'Gyeongsangnam-do': 'Changwon',
    'St. Gallen': 'St. Gallen',
    'Drenthe': 'Assen',
    'State of Ceara': 'Fortaleza',
    'Marche': 'Ancona',
    'West Pomeranian Voivodeship': 'Szczecin',
    'Friuli-Venezia Giulia': 'Trieste',
    'Decentralized Administration of Attica': 'Athens',
    'Viken': 'Drammen',
    'Lower Austria': 'St. Pölten',
    'State of Pernambuco': 'Recife',
    'Veracruz': 'Veracruz',
    'Flanders': 'Antwerp',
    'Kuyavian-Pomeranian Voivodeship': 'Bydgoszcz',
    'Krasnodar Krai': 'Krasnodar',
    'Central Bohemian Region': 'Prague',
    'State of Espirito Santo': 'Vitória',
    'Abruzzo': 'Pescara',
    'Region Zealand': 'Roskilde',
    'Maryland': 'Baltimore',
    'South District': 'Ashdod',
    'Normandy': 'Rouen',
    'Iowa': 'Des Moines',
    'Jerusalem District': 'Jerusalem',
    'Wisconsin': 'Milwaukee',
    'Gyeongsangbuk-do': 'Gyeongju',
    'Puebla': 'Puebla',
    'Friesland': 'Leeuwarden',
    'Bourgogne-Franche-Comte': 'Dijon',
    'Chungcheongbuk-do': 'Cheongju',
    'Sonora': 'Hermosillo',
    'Penang': 'George Town',
    'Federation of Bosnia and Herzegovina': 'Sarajevo',
    'Trentino-South Tyrol': 'Bolzano',
    'Cordoba': 'Córdoba',
    'Hyogo': 'Kobe',
    'Brandenburg': 'Potsdam',
    'Baden-Wurttemberg': 'Stuttgart',
    'Setubal': 'Setúbal',
    'Guanajuato': 'Guanajuato',
    'Kocaeli': 'Izmit',
    'Groningen': 'Groningen',
    'Muscat Governorate': 'Muscat',
    'Giza Governorate': 'Giza',
    'Arkansas': 'Little Rock',
    'Community of Madrid': 'Madrid',
    'Skane County': 'Malmö',
    'South Moravian Region': 'Brno',
    'Podkarpackie Voivodeship': 'Rzeszów',
    'Minnesota': 'Minneapolis',
    'Utrecht': 'Utrecht',
    'North Rhine-Westphalia': 'Cologne',
    'Indiana': 'Indianapolis',
    'Louisiana': 'New Orleans',
    'Odisha': 'Bhubaneswar',
    'Newfoundland and Labrador': 'St. John\'s',
    'Canton of Bern': 'Bern',
    'Bihar': 'Patna',
    'Utah': 'Salt Lake City',
    'Lower Saxony': 'Hanover',
    'Guayas': 'Guayaquil',
    'Gangwon-do': 'Gangneung',
    'Brittany': 'Rennes',
    'Mississippi': 'Jackson',
    'Pays de la Loire': 'Nantes',
    'Nuevo Leon': 'Monterrey',
    'New Hampshire': 'Concord',
    'South Dakota': 'Sioux Falls',
    'Chhattisgarh': 'Raipur',
    'Uttarakhand': 'Dehradun'
}



# device columns

- for many cases we assumed devices, brands and os versions of 2021 as fill method

## device mobile brand

In [59]:

# Fill 'device_mobile_brand_name' with 'PC' where the conditions are met
data.loc[(data['device_operating_system'] == 'Windows') & (data['device_category'] == 'desktop'), 'device_mobile_brand_name'] = 'PC'
data.loc[(data['device_operating_system'] == 'Web') & (data['device_category'] == 'desktop') & ((data['device_mobile_model_name'].isin(['Chrome','Edge','Firefox']))), 'device_mobile_brand_name'] = 'PC'
data.loc[(data['device_mobile_brand_name'] == 'Microsoft') & (data['device_category'] == 'desktop'), 'device_mobile_brand_name'] = 'PC'


data['device_mobile_brand_name'].value_counts()

device_mobile_brand_name
Apple        304160
PC           225248
Google        54986
<Other>       52490
Samsung       51922
Xiaomi        16178
Huawei         9536
Mozilla         624
Microsoft        80
Name: count, dtype: int64

## device_mobile_model_name 

In [60]:


data.loc[(data['device_mobile_brand_name'] == 'Samsung'), 'device_mobile_model_name'] = 'Galaxy S21'

data.loc[(data['device_mobile_brand_name'] == 'Xiaomi'), 'device_mobile_model_name'] = 'Mi 11'

data.loc[(data['device_mobile_brand_name'] == 'Huawei'), 'device_mobile_model_name'] = 'P50'

data.loc[(data['device_mobile_brand_name'] == 'Apple')& (data['device_category'] == 'desktop'), 'device_mobile_model_name'] = 'Macintosh'

data.loc[(data['device_mobile_brand_name'] == 'PC')& (data['device_mobile_model_name'] == 'Chrome'), 'device_mobile_model_name'] = 'PC'


data['device_mobile_model_name'].value_counts()

device_mobile_model_name
PC            196709
iPhone        148257
Macintosh     144731
<Other>        69099
Galaxy S21     51922
ChromeBook     38915
Mi 11          16178
Edge           15284
iPad            9728
P50             9536
Firefox         9387
Chrome          2076
Pixel 4 XL      1758
Pixel 3         1626
Safari            18
Name: count, dtype: int64

## device_operating_system

In [61]:

# Chrome OS - ensure 'desktop' is correctly spelled
data.loc[(data['device_mobile_model_name'] == 'ChromeBook') & (data['device_category'] == 'desktop'), 'device_operating_system'] = 'ChromeOS'

# iOS - for iPhone and iPad
data.loc[data['device_mobile_model_name'].isin(['iPhone', 'iPad']) | (data['device_mobile_brand_name'] == 'Apple') | ((data['device_mobile_model_name'] == 'Apple') & (data['device_category'].isin(['mobile','tablet']))), 'device_operating_system'] = 'iOS'

# Android - for specified brands
android_brands = ['Xiaomi', 'Huawei', 'Samsung']
data.loc[data['device_mobile_brand_name'].isin(android_brands), 'device_operating_system'] = 'Android'
data.loc[(data['device_mobile_brand_name'] == 'Google') & (data['device_category'].isin(['mobile','tablet'])), 'device_operating_system'] = 'Android'

# macOS
data.loc[(data['device_mobile_brand_name'] == 'Apple') & (data['device_category'] == 'desktop'), 'device_operating_system'] = 'MacOS'

# Windows
data.loc[(data['device_operating_system'] == 'Web') & (data['device_category'] == 'desktop') & ((data['device_mobile_brand_name'] == 'PC')), 'device_operating_system'] = 'Windows'
data.loc[(data['device_operating_system'] == 'Web') & (data['device_category'] == 'desktop') & ((data['device_mobile_brand_name'] == 'Mozilla')), 'device_operating_system'] = 'Windows'
data.loc[(data['device_category'] == 'desktop') & ((data['device_mobile_brand_name'] == 'Microsoft')), 'device_operating_system'] = 'Windows'


data.groupby(['device_category','device_operating_system', 'device_mobile_brand_name']).agg(
    unique_event_count=('event_timestamp', 'nunique')
).sort_values('unique_event_count',ascending=False).reset_index()


,device_category,device_operating_system,device_mobile_brand_name,unique_event_count
0,desktop,Windows,PC,167207
1,mobile,iOS,Apple,111783
2,desktop,MacOS,Apple,107747
3,mobile,Android,Samsung,36350
4,desktop,ChromeOS,Google,29097
5,mobile,Web,<Other>,19459
6,mobile,Android,<Other>,14552
7,mobile,Android,Xiaomi,11954
8,mobile,Android,Google,10546
9,tablet,iOS,Apple,7229


## device_operating_system_version

In [62]:
# replace all string characters and keep float values
data['device_operating_system_version'] = data['device_operating_system_version'].str.extract(r'(\d+\.\d+|\d+)')


In [63]:
# for chrome os consider same browser version
# https://chromereleases.googleblog.com/2021/
data.loc[(data['device_operating_system'] == 'ChromeOS') & (data['device_operating_system_version'].isnull()), 'device_operating_system_version'] = data['device_web_info_browser_version']


data['device_operating_system_version'].value_counts()


device_operating_system_version
10         261483
10.15      101299
14.3        31733
14.2        30791
86.0        18738
87.0        15906
9           10444
11.1        10023
7            5742
<Other>      4271
Name: count, dtype: int64

In [64]:
data['device_operating_system_version'].value_counts()

device_operating_system_version
10         261483
10.15      101299
14.3        31733
14.2        30791
86.0        18738
87.0        15906
9           10444
11.1        10023
7            5742
<Other>      4271
Name: count, dtype: int64

## device_language

In [65]:
data.groupby(['geo_continent','geo_country','device_language']).agg(
    unique_event_count=('event_timestamp', 'nunique')
).sort_values('unique_event_count',ascending=False).reset_index()

,geo_continent,geo_country,device_language,unique_event_count
0,Americas,United States,en-us,87189
1,Asia,India,en-us,18500
2,Americas,Canada,en-us,15218
3,Americas,United States,en-gb,13904
4,Americas,United States,en,7016
...,...,...,...,...
797,Asia,Macao,en-ca,1
798,Asia,Mongolia,fr,1
799,Asia,Oman,en,1
800,Asia,Oman,fr,1


In [66]:
data.groupby(['geo_continent','geo_country']).agg(
    null_device_language_count=('device_language', lambda x: x.isna().sum())
).sort_values('null_device_language_count',ascending=False).reset_index()

,geo_continent,geo_country,null_device_language_count
0,Americas,United States,147594
1,Asia,India,29182
2,Americas,Canada,25645
3,Europe,United Kingdom,9780
4,Europe,Spain,6796
...,...,...,...
103,Americas,Honduras,38
104,Europe,Malta,38
105,Americas,Trinidad & Tobago,25
106,Europe,Bosnia & Herzegovina,21


In [67]:
data['device_language'].value_counts()

device_language
en-us    260636
en-gb     40875
zh        19408
en        19295
en-ca     14459
fr         9805
es-es      9325
de         5226
ko         4426
Name: count, dtype: int64

## device_web_info_browser

In [68]:
data.loc[(data['device_web_info_browser'] == 'Android Webview'), 'device_web_info_browser'] = "Chrome"


data['device_web_info_browser'].value_counts()

device_web_info_browser
Chrome     497582
Safari     170118
<Other>     19337
Edge        15546
Firefox     12641
Name: count, dtype: int64

## device_web_info_browser_version

In [69]:
data['device_web_info_browser_version'].value_counts()

device_web_info_browser_version
87.0       262740
86.0       152984
<Other>    127123
14.0       125390
604         17794
13.1        16431
84.0         4243
83.0         3615
13.0         2914
82.0         1940
14.1           31
87             14
86              4
14.2            1
Name: count, dtype: int64

# Traffic Columns

## session counts

In [70]:
data.groupby(['traffic_source_medium','traffic_source_source']).agg(
    unique_session_count=('session_id', 'nunique')
).sort_values('unique_session_count',ascending=False).reset_index()

,traffic_source_medium,traffic_source_source,unique_session_count
0,organic,google,47208
1,(none),(direct),34987
2,<Other>,<Other>,21735
3,referral,<Other>,14747
4,referral,shop.googlemerchandisestore.com,12422
5,(data deleted),(data deleted),9596
6,cpc,google,6520
7,organic,<Other>,4246
8,(data deleted),<Other>,167
9,referral,(data deleted),2


## null counts

In [71]:
data.groupby(['traffic_source_medium']).agg(
    null_count=('traffic_source_source', lambda x: x.isna().sum())
).sort_values('null_count',ascending=False).reset_index()

,traffic_source_medium,null_count
0,(data deleted),0
1,(none),0
2,<Other>,0
3,cpc,0
4,organic,0
5,referral,0


In [72]:
data.groupby(['traffic_source_source']).agg(
    null_count=('traffic_source_medium', lambda x: x.isna().sum())
).sort_values('null_count',ascending=False).reset_index()

# no null count on

,traffic_source_source,null_count
0,(data deleted),0
1,(direct),0
2,<Other>,0
3,google,0
4,shop.googlemerchandisestore.com,0


## remove parenthesis ()

In [73]:
data['traffic_source_source'] = data['traffic_source_source'].str.replace(r'\(|\)', '', regex=True).str.strip()
data['traffic_source_medium'] = data['traffic_source_medium'].str.replace(r'\(|\)', '', regex=True).str.strip()


data.groupby(['traffic_source_medium','traffic_source_source']).agg(
    unique_session_count=('session_id', 'nunique')
).sort_values('unique_session_count',ascending=False).reset_index()

,traffic_source_medium,traffic_source_source,unique_session_count
0,organic,google,47208
1,none,direct,34987
2,<Other>,<Other>,21735
3,referral,<Other>,14747
4,referral,shop.googlemerchandisestore.com,12422
5,data deleted,data deleted,9596
6,cpc,google,6520
7,organic,<Other>,4246
8,data deleted,<Other>,167
9,referral,data deleted,2


## Replace values

- bad referral naming from shop.googlemerchandisestore.com means badly tracked and we should consider direct traffic instead
- medium =none is referring to direct traffic and we will use the same name to not confuse with null values
- data deleted is most likely paid campaign by google to avoid confidential data exposure so we will replace with cpc / google as well


In [74]:

# data.loc[(data['traffic_source_source'] == 'shop.googlemerchandisestore.com') & (data['traffic_source_medium'] == 'referral'), 'traffic_source_source'] = 'direct'
# data.loc[(data['traffic_source_source'] == 'shop.googlemerchandisestore.com') & (data['traffic_source_medium'] == 'referral'), 'traffic_source_medium'] = 'direct'
# source is merchandisestore.com then direct (bad parameter)
data.loc[(data['traffic_source_source'] == 'shop.googlemerchandisestore.com') & (data['traffic_source_medium'] == 'referral'), ['traffic_source_medium', 'traffic_source_source']] = ['direct','direct']


# referral traffic
data.loc[(data['traffic_source_medium'] == 'referral'), ['traffic_source_medium', 'traffic_source_source']] = ['referral','referral_link']

# google organic
data.loc[(data['traffic_source_source'].isnull()) & (data['traffic_source_medium']== 'organic'),  ['traffic_source_medium', 'traffic_source_source']] = ['organic','google']


# when we have direct traffic it is direct traffic
data.loc[(data['traffic_source_source'] == 'direct'), 'traffic_source_medium'] = 'direct'

# data deleted is paid campaign cpc by google
# data.loc[(data['traffic_source_source'] == 'data deleted') | (data['traffic_source_medium'] == 'data deleted'), 'traffic_source_medium'] = 'cpc'
# data.loc[(data['traffic_source_source'] == 'data deleted') | (data['traffic_source_medium'] == 'data deleted'), 'traffic_source_source'] = 'google'
data.loc[(data['traffic_source_source'] == 'data deleted') | (data['traffic_source_medium'] == 'data deleted'), ['traffic_source_medium', 'traffic_source_source']] = ['cpc', 'google']


# full null values are direct
data.loc[(data['traffic_source_source'].isnull()) & (data['traffic_source_medium'].isnull()),  ['traffic_source_medium', 'traffic_source_source']] = ['direct','direct']
# data.loc[(data['traffic_source_source'].isnull()) & (data['traffic_source_medium'].isnull()), 'traffic_source_medium'] = 'direct'



# data = fill_nulls_based_on_top_value_multiple_columns(data, 'traffic_source_source', ['device_category','geo_country'])
# data = fill_nulls_based_on_top_value_multiple_columns(data, 'traffic_source_medium', ['device_category','geo_country','traffic_source_source','traffic_source_medium'])


data.groupby(['traffic_source_source','traffic_source_medium']).agg(
    unique_session_count=('session_id', 'nunique')
).sort_values('unique_session_count',ascending=False).reset_index()

,traffic_source_source,traffic_source_medium,unique_session_count
0,direct,direct,47333
1,google,organic,47208
2,<Other>,<Other>,21735
3,google,cpc,16273
4,referral_link,referral,14749
5,<Other>,organic,4246
6,<Other>,cpc,1


# Page columns

In [75]:
df_page_ref=data.groupby(['page_location']).agg(
    unique_session_count=('session_id', 'nunique')
).sort_values('unique_session_count',ascending=False).reset_index()

In [76]:
# remove domain name for better readability
data['page_location'] = data['page_location'].str.replace(r'shop.googlemerchandisestore.com/store.html', 'shop.googlemerchandisestore.com/').str.strip()
data['page_location'] = data['page_location'].str.replace(r'+', ' ').str.strip()
data['page_location'] = data['page_location'].str.replace(r'https://', '').str.strip()
data['page_location'] = data['page_location'].str.replace(r'http://', '').str.strip()
data['page_location'] = data['page_location'].str.replace(r'www.', '').str.strip()


df_pages_agg=data.groupby(['page_location','page_title']).agg(
    unique_session_count=('session_id', 'nunique')
).sort_values('unique_session_count',ascending=False).reset_index()

df_pages_agg

,page_location,page_title,unique_session_count
0,shop.googlemerchandisestore.com/,Home,53663
1,googlemerchandisestore.com/,Google Online Store,21096
2,shop.googlemerchandisestore.com/Google Redesig...,Apparel | Google Merchandise Store,16131
3,shop.googlemerchandisestore.com/Google Redesig...,Men's / Unisex | Apparel | Google Merchandise ...,15251
4,shop.googlemerchandisestore.com/Google Redesig...,Sale | Google Merchandise Store,12084
...,...,...,...
1334,shop.googlemerchandisestore.com/eco/Android Na...,Page Unavailable,1
1335,shop.googlemerchandisestore.com/eco/Eco-Jersey...,Page Unavailable,1
1336,shop.googlemerchandisestore.com/eco/Youth Blac...,Page Unavailable,1
1337,shop.googlemerchandisestore.com/eco/youtube or...,Page Unavailable,1


## split categories in page paths

In [77]:
# Step 1: Split 'page_location' into 4 parts (max)
split_columns = data['page_location'].str.split('/', n=4, expand=True)

# Step 2: Assign the first three parts to new columns (ignore the first empty part if there is a leading '/')
data['domain'] = split_columns[0] # url_domain 
data['page_path_1'] = split_columns[1].replace('', pd.NA)
data['page_path_2'] = split_columns[2].replace('', pd.NA)
data['page_path_3'] = split_columns[3].replace('', pd.NA)




df_pages_total=data[['page_title','page_location','page_path_1','page_path_2','page_path_3']]


## fill length page path with page title

In [78]:
data['page_path_1'] = data.apply(
    lambda row: row['page_title'] if pd.isna(row['page_path_2']) else row['page_path_1'], 
    axis=1
)

df_pages_total=data[['page_title','page_location','page_path_1','page_path_2','page_path_3']]


## fill the other page path levels with previous page path column

This will allow for hierarchical encoding without sacrificing columns or rows

In [79]:
# Fill 'page_path_2' with 'page_path_1' if 'page_path_2' is null
data['page_path_2'] = data.apply(
    lambda row: row['page_path_1'] if pd.isna(row['page_path_2']) else row['page_path_2'], 
    axis=1
)

# Fill 'page_path_3' with 'page_path_2' if 'page_path_3' is null
data['page_path_3'] = data.apply(
    lambda row: row['page_path_2'] if pd.isna(row['page_path_3']) else row['page_path_3'], 
    axis=1
)


df_pages_total=data[['page_title','page_location','page_path_1','page_path_2','page_path_3']]

In [80]:

df_pages_agg=data.groupby(['page_title','domain','page_location','page_path_1','page_path_2','page_path_3']).agg(
    unique_session_count=('session_id', 'nunique')
).sort_values('unique_session_count',ascending=False).reset_index()

df_pages_agg

,page_title,domain,page_location,page_path_1,page_path_2,page_path_3,unique_session_count
0,Home,shop.googlemerchandisestore.com,shop.googlemerchandisestore.com/,Home,Home,Home,53663
1,Google Online Store,googlemerchandisestore.com,googlemerchandisestore.com/,Google Online Store,Google Online Store,Google Online Store,21096
2,Apparel | Google Merchandise Store,shop.googlemerchandisestore.com,shop.googlemerchandisestore.com/Google Redesig...,Google Redesign,Apparel,Apparel,16131
3,Men's / Unisex | Apparel | Google Merchandise ...,shop.googlemerchandisestore.com,shop.googlemerchandisestore.com/Google Redesig...,Google Redesign,Apparel,Mens,15251
4,Sale | Google Merchandise Store,shop.googlemerchandisestore.com,shop.googlemerchandisestore.com/Google Redesig...,Google Redesign,Clearance,Clearance,12084
...,...,...,...,...,...,...,...
1312,Page Unavailable,shop.googlemerchandisestore.com,shop.googlemerchandisestore.com/google redesig...,google redesign,accessories,google cork passport holder,1
1313,Page Unavailable,shop.googlemerchandisestore.com,shop.googlemerchandisestore.com/google redesig...,google redesign,accessories,google felt strap keyring,1
1314,Page Unavailable,shop.googlemerchandisestore.com,shop.googlemerchandisestore.com/google redesig...,google redesign,accessories,google inverted umbrella,1
1315,Page Unavailable,shop.googlemerchandisestore.com,shop.googlemerchandisestore.com/google redesig...,google redesign,accessories,google kirkland campus sticker,1


# Final df without relevant columns

In [81]:
data_final=data.drop(['page_title','page_location','session_id','user_pseudo_id'],axis=1) 

In [82]:
data.shape

(715224, 42)

In [83]:
data_final.to_csv('df_merch_pre_proc.csv',index=False)

# df_merch_pre_proc.csv
# df_merch_profile.csv
# df_merch_data_prep.csv